In [68]:
import requests, pandas as pd

In [74]:
# The base URL to use for the request
base_url = "https://api.carbonintensity.org.uk/regional/intensity/"
from_date = '2024-08-25T00:00Z'
to_date = '2024-08-26T00:00Z'
postcode = "BN3"


In [75]:
# Make the GET request
response = requests.get(base_url+from_date+"/"+to_date+"/postcode/"+postcode)

# Check if the request was successful
if response.status_code == 200:
    print("Response received: OK")
else:
    print(f"Error: {response.status_code}")

Response received: OK


In [76]:
def flatten(dictionary):
    result = {}

    for (k,v) in dictionary.items():
        if isinstance(v, dict):
            for (k2,v2) in v.items():
                result[k+"_"+k2] = v2
        elif isinstance(v, list):
            for i in v:
                result[k+"_"+i['fuel']] = i['perc']
        else:
            result[k] = v

    return result

In [77]:
data = response.json()
data['data']['data'] = list(map(flatten, data['data']['data']))

In [80]:
df = pd.json_normalize(data['data'], 'data', meta=['regionid', 'shortname', 'postcode'])
df.head()

,from,to,intensity_forecast,intensity_index,generationmix_biomass,generationmix_coal,generationmix_imports,generationmix_gas,generationmix_nuclear,generationmix_other,generationmix_hydro,generationmix_solar,generationmix_wind,regionid,shortname,postcode
0,2024-08-24T23:30Z,2024-08-25T00:00Z,82,low,1.7,0,0.0,20.2,9.1,0,5.0,0.0,64.0,14,South East England,BN3
1,2024-08-25T00:00Z,2024-08-25T00:30Z,79,low,1.4,0,0.0,19.5,9.9,0,4.1,0.0,65.1,14,South East England,BN3
2,2024-08-25T00:30Z,2024-08-25T01:00Z,78,low,1.2,0,0.0,19.5,10.9,0,3.2,0.0,65.2,14,South East England,BN3
3,2024-08-25T01:00Z,2024-08-25T01:30Z,85,low,1.3,0,0.0,21.3,11.0,0,2.5,0.0,63.9,14,South East England,BN3
4,2024-08-25T01:30Z,2024-08-25T02:00Z,80,low,1.5,0,0.0,20.0,11.3,0,2.0,0.0,65.2,14,South East England,BN3


In [81]:
df.to_csv('data.csv')